In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [60]:
class Network(object):
    '''
    Abbreviations:
    W : Weight matrix for each layer.
    b : Bias vector for each layer.
    z : Neuron vector before applying sigmoid function (unsmoothed).
    a : Network input vector / layer activation vector.
    y : Expected network output vector.

    Notes on indeces:
        As calculations are performed from first hidden layer
        onward and Python is zero-indexed, we denote this first
        non-input layer as zero.
    '''

    @staticmethod
    def sigmoid(z):
        '''Calculates sigmoid of z.'''
        return 1 / (1 + np.e**(-z))


    @staticmethod
    def sigmoid_derivative(z):
        '''Evaluates derivate of sigmoid at z.'''
        half_exponent = np.e**(z / 2)
        return (half_exponent / (half_exponent + 1))**2
    
    
    def __init__(self, size):
        self.number_of_hidden_layers = len(size) - 2
        self.size = size
        self.biases = [np.random.randn(y, 1) for y in size[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(size[:-1], size[1:])]


    def check_input(self, a):
        '''Checks validity of input vector a.'''
        if not isinstance(a, np.ndarray):
            a = np.transpose(np.array(a))
        if a.shape != (self.size[0], 1):
            raise ValueError(f'Input vector must be of shape ({self.size[0]}, 1). Received shape {a.shape}.')
        return a

    def check_output(self, y):
        '''Checks validity of expected output vector y.'''
        if not isinstance(y, np.ndarray):
            y = np.transpose(np.array(y))
        if y.shape != (self.size[-1], 1):
            raise ValueError(f'Output vector must be of shape ({self.size[-1]}, 1). Received shape {y.shape}.')
        return y

    
    def front_propagate(self, a):
        '''Front propagates input vector a.'''
        a = self.check_input(a)
        
        for W, b in zip(self.weights, self.biases):
            a = self.sigmoid(np.dot(W, a) + b)
        return a


    def calculate_cost(self, data):
        '''
        Calculates cost with respect to (input vector, expected output vector)
        pairs in data.

        Parameters
        ----------
        data : array_like
            Array containing (input vector, expected output vector) pairs.

        Returns
        -------
        cost : float
            Average of evaluated cost at each data pair.
        '''
        cost = 0
        data_length = len(data)
        
        for a, y in data:
            y = self.check_output(y)            
            difference = self.front_propagate(a) - y
            cost += float(np.dot(np.transpose(difference), difference)) / data_length
        return cost


    ## We may return a list of unsmoothed activations and combine with
    ## calculate neuron activations.
    def feedforward(self, a, n):
        '''
        Front propagates input vector a up to (n - 1)th layer
        then calculates z (unsmoothed neuron vector) of nth layer.
        
        Parameters
        ----------
        a : array_like
            Network input vector.
        n : int
            Number of layer to which propagate.

        Returns
        -------
        z : Unsmoothed nth layer activations.
        '''
        a = self.check_input(a)

        if n > 0:
            for W, b in zip(self.weights[:n], self.biases[:n]):
                a = self.sigmoid(np.dot(W, a) + b)
        z = np.dot(self.weights[n], a) + self.biases[n]
        return z

    
    def calculate_neuron_activations(self, a):
        '''
        Calculates neuron activations at each layer from input vector a.
        Returns list containing 2D column vectors from each layer activations.

        Parameters
        ----------
        a : array_like
            Network input vector.

        Return
        ------
        neuron_activations : list of ndarray
            List of 2D arrays. Each 2D array is a column vector containing layer
            activations.
        '''
        a = self.check_input(a)
        
        neuron_activations = list([a])
        for W, b in zip(self.weights, self.biases):
            a = np.dot(W, a) + b
            neuron_activations.append(a)
        return neuron_activations

    
    def calculate_output_partials(self, a, y):
        '''
        Calculates partials of cost function (MSE) with respect to last
        layer unsmoothed neuron activations.
        
        Parameters
        ----------
        a : array_like
            Network input vector.
        y : array_like
            Expected network output.

        Returns
        -------
        output_error : ndarray
            2D array, column vector. Partials of cost function with respect
            to last layer unsmoothed neuron activations.
        '''
        a = self.front_propagate(a)
        y = self.check_output(y)
        
        sig_derivative = self.sigmoid_derivative(a)
        gradient_cost = a - y
        output_partials = np.multiply(gradient_cost, sig_derivative)
        return output_partials


    ## Instead of feedforwarding each z inside, we may pass a list of already
    ## calculated z.
    def calculate_layer_partials(self, a, n, next_partials):
        '''
        Calculates nth layer partials from (n + 1)th layer partials.

        Parameters
        ----------
        a : array_like
            Network input vector.
        n : int
            Number of layer at which partials are calculated.
        next_partials : ndarray
            2D array, column vector. Cost function partials with respect to unsmoothed
            neurons from (n + 1)th layer.

        Returns
        -------
        n_partials : ndarray
            2D array, column vector. Cost function partials with respect to unsmoothed
            neurons from nth layer.
        '''
        z = self.feedforward(a, n)
        sig_derivative = self.sigmoid_derivative(z)
        weighted_partials = np.dot(np.transpose(self.weights[n + 1]), next_partials)
        n_partials = np.multiply(weighted_partials, sig_derivative)
        return n_partials


    ## Can include in back propagate direclty.
    def calculate_weight_partials(self, n_partials, prev_a):
        '''
        Calculates matrix containing each of the cost function partials
        with respects to weights on nth layer.

        Parameters
        ----------
        n_partials : ndarray
            2D array, column vector. Cost function partials with respect to unsmoothed
            neurons from nth layer.

        prev_ a : ndarray
            2D array, column vector. Neuron activations from (n - 1)th layer.

        Returns
        -------
        n_weight_partials : ndarray
            2D array, matrix. Cost function partials with respect to weights on
            nth layer.
        '''
        n_weight_partials = np.dot(n_partials, np.transpose(prev_a))
        return n_weight_partials

    ## We can combine the two for loops inside.
    def back_propagate(self, a, y):
        '''
        Calculates gradient of the cost function with respect to
        weights and biases, evaluated at input vector a and expected
        output vector y. Returns 3D array of column vectors of each layer
        of bias partials and 3D array of matricies of each layer weights partials.

        Parameters
        ----------
        a : array_like
            Network input vector.
        y : array_like
            Network expected output vector.

        Returns
        -------
        bias_partials : list of ndarray
            List of 2D arrays (column vectors). Partials of the cost
            function with respect to biases. Each column vector holds
            partials with respect to biases of ith layer.
        weight_partials : list of ndarray
            List of 2d ndarrays, array of 2D matrices. Partials of the cost
            function with respect to weights. Each 2D matrix holds partials
            with respect to weights of ith layer.
        '''
        output_partials = self.calculate_output_partials(a, y)
        # Partials with respect to unsmoothed neurons in each layer.
        z_partials = []
        # We'll add partials in reverse order, starting from last layer
        # and working our way to the first hidden layer.
        z_partials.append(output_partials)
        # As Python is zero indexed, we substract 1 from the number of hiden layers.
        for n in range(self.number_of_hidden_layers - 1, -1, -1):
            n_partials = self.calculate_layer_partials(a, n, z_partials[-1])
            z_partials.append(n_partials)
        z_partials.reverse()

        # Partials with respect to unsmoothed neurons are
        # the same as partials with respect to biases.
        bias_partials = z_partials

        # Neuron activations for each layer.
        neuron_activations = self.calculate_neuron_activations(a)
        weight_partials = []
        for z_partial, neuron in zip(z_partials, neuron_activations[:-1]):
            n_weight_partials = self.calculate_weight_partials(z_partial, neuron)
            weight_partials.append(n_weight_partials)

        return weight_partials, bias_partials


    def update(self, a, y, learning_rate):
        '''
        Updates neural network weights and biases using gradient descent.

        Parameters
        ----------
        a : array_like
            Network input vector.
        y : array_like
            Network expected output vector.
        learning_rate : float
            Learning rate of neural network.
        Returns
        -------
        None
        '''
        weight_partials, bias_partials = self.back_propagate(a, y)
        new_weights = []
        new_biases = []
        for W, b, W_p, b_p in zip(self.weights, self.biases, weight_partials, bias_partials):
            new_weights.append(W - learning_rate * W_p)
            new_biases.append(b - learning_rate * b_p)

        self.weights = new_weights
        self.biases = new_biases
            

    def SGD(self, mb_size, learning_rate, training_data, test_data = None):
        '''
        Applies stochastic gradient descent to neural network by using
        batches of size mb_size. Test data may be provided to track
        cost function after every epoch.

        Parameters
        ----------
        mb_size : int
            Size of mini-batch.
        learning_rate : float
            Learning rate of neural network.
        training_data : array_like
            Array containing (input vector, expected output vector) pairs.
        test_data : array_like
            Optional. Array containing (input vector, expected output vector) pairs
            used to calculate cost function after every epoch.

        Returns
        -------
        costs : ndarray
            Array containing cost calculate from test_data after each mini-batch.
        '''
        np.random.shuffle(training_data)

        # Number of mini-batches that can be generated from training data.
        n_of_mb = int(np.ceil(len(training_data) / mb_size))
        # Array to save test data costs.
        costs = np.zeros(n_of_mb)
        for i in range(n_of_mb):
            for a, y in training_data[i * mb_size:(i + 1) * mb_size]:
                # Instead of taking the average of partials and then updating,
                # we can directly update with each partials scaled by 1 / mb_size.
                self.update(a, y, learning_rate / mb_size)
            if test_data != None:
                costs[i] = self.calculate_cost(test_data)

        if test_data != None:
            return costs
                

    def print_parameters(self):
        biases_shapes = [b.shape for b in self.biases]
        weights_shapes = [W.shape for W in self.weights]
        print(f'layers: {self.size}, biases: {biases_shapes}, weights: {weights_shapes}')

In [61]:
np.random.seed(1)
net_size = [20, 300, 20]
net = Network(net_size)
net.print_parameters()

layers: [20, 300, 20], biases: [(300, 1), (20, 1)], weights: [(300, 20), (20, 300)]


In [62]:
# Training data / test data
# Percentage to be treated as traning_data
ratio = 0.9
data_size = 100000
input_vectors = [net.sigmoid(np.random.normal(0, 1, (20, 1))) for n in range(data_size)]
output_vectors = [np.cos(a) for a in input_vectors]
data = []
for a, y in zip(input_vectors, output_vectors):
    data.append((a, y))
np.random.shuffle(data)
training_data = data[:int(0.9 * data_size)]
test_data = data[int(0.9 * data_size):]

In [63]:
print(net.calculate_cost(test_data))
mb_size = 10000
lr = 20
costs = net.SGD(mb_size, lr, training_data, test_data)
print(costs)

6.460410944248395
[0.25371292 0.14277961 0.11715329 0.11316049 0.10896235 0.10353365
 0.10338407 0.10171091 0.10249596]


In [65]:
np.average(net.front_propagate(input_vectors[100]) - input_vectors[30])

0.3587889589606101

In [66]:
## Tests
B = np.transpose(np.array([[1, 2, 3, 4, 5]]))
A = np.transpose(np.array([[10, 100, 1000]]))
A = np.transpose(A)

np.dot(B, A)

array([[  10,  100, 1000],
       [  20,  200, 2000],
       [  30,  300, 3000],
       [  40,  400, 4000],
       [  50,  500, 5000]])

In [67]:
A = [1, 2, 3, 4]
B = ["A", "B", "C", "D"]
for i, c in enumerate(zip(A, B)):
    print(i, c)

0 (1, 'A')
1 (2, 'B')
2 (3, 'C')
3 (4, 'D')
